 **Sudoku Solver** - quick sudoku solver in Python, using PuLP
=================

### A sudoku solver for any board you find. Have a go! 🎱

This is the notebook for the repository [sudoku-solver](https://github.com/voaneves/sudoku-solver), in which you could test the execution with GPU/CPU for a sudoku solver. 

Let's begin using PuLP for Linear Programming in order to solve the sudoku.

# Table of Contents

[1. Imports](#imports)

[2. Add constraints](#constraints)

[3. Extract and print a solution](#extract)

[4. Solver](#solver)

[5. Random board creator](#random)

[6. Testing](#testing)

## 1. Imports <a name="imports"></a>

### 1.1. Install dependencies

In [1]:
%pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1.2. Import the packages

In [2]:
import pulp as plp

## 2. Add constraints <a name="constraints"></a>

### 2.1. Default Constraints

Considering sudoku as a linear problem, the following constraints are necessary:

* **Constraint 1**: Each cell should be filled with a single value between 1 and 9  
* **Constraint 2**: Each row should contain every number from 1 to 9 once 
* **Constraint 3**: Each column should contain every number from 1 to 9 once 
* **Constraint 4**: Each 3x3 grid, starting from top left, should contain every number from 1 to 9 once

In [3]:
def add_default_constraints(prob, target_vars, rows, cols, grids, values):
    # only one value per cell
    for row in rows:
        for col in cols:
                prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[row][col][value] for value in values]),
                                                    sense = plp.LpConstraintEQ,
                                                    rhs = 1,
                                                    name = f"constraint_sum_{row}_{col}"))
                
    # each row have numbers 1 to 9 only once        
    for row in rows:
        for value in values:
            prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[row][col][value]*value  for col in cols]),
                                                sense = plp.LpConstraintEQ,
                                                rhs = value,
                                                name = f"constraint_uniq_row_{row}_{value}"))
            
    # each column have numbers 1 to 9 only once         
    for col in cols:
        for value in values:
            prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[row][col][value]*value  for row in rows]),
                                                sense = plp.LpConstraintEQ,
                                                rhs = value,
                                                name = f"constraint_uniq_col_{col}_{value}"))

    # each grid have numbers 1 to 9 only once        
    for grid in grids:
        grid_row  = int(grid/3)
        grid_col  = int(grid%3)

        for value in values:
            prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[grid_row*3+row][grid_col*3+col][value]*value  for col in range(0,3) for row in range(0,3)]),
                                        sense = plp.LpConstraintEQ,
                                        rhs = value,
                                        name = f"constraint_uniq_grid_{grid}_{value}"))

### 2.2. Diagonal constraints

For diagonals sudoku, there's an additional constraint, as follows:

* **Constraint 5**: Each diagonal should contain every number from 1 to 9 once

In [4]:
def add_diagonal_sudoku_constraints(prob, target_vars, rows, cols, values):
        # Constraint from top-left to bottom-right - numbers 1 - 9 should not repeat
        for value in values:
                prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[row][row][value]*value  for row in rows]),
                                                    sense = plp.LpConstraintEQ,
                                                    rhs = value,
                                                    name = f"constraint_uniq_diag1_{value}"))

        # Constraint from top-right to bottom-left - numbers 1 - 9 should not repeat
        for value in values:
                prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[row][len(rows)-row-1][value]*value  for row in rows]),
                                                    sense = plp.LpConstraintEQ,
                                                    rhs = value,
                                                    name = f"constraint_uniq_diag2_{value}"))


### 2.3. Prefilled values as constraints

It's important to set the giver board values as constants. 

* **Constraint 6**: Fill the prefilled cells as constraints to the LP problem.

In [5]:
def add_prefilled_constraints(prob, input_sudoku, target_vars, rows, cols, values):
    for row in rows:
        for col in cols:
            if(input_sudoku[row][col] != 0):
                prob.addConstraint(plp.LpConstraint(e = plp.lpSum([target_vars[row][col][value]*value  for value in values]), 
                                                    sense = plp.LpConstraintEQ, 
                                                    rhs = input_sudoku[row][col],
                                                    name = f"constraint_prefilled_{row}_{col}"))

## 3. Extract & Print Solution <a name="extract"></a>

### 3.1. Extract solution from the target variable to a list array

In [6]:
def extract_solution(target_vars, rows, cols, values):
    solution = [[0 for col in cols] for row in rows]
    grid_list = []

    # extract the solution
    for row in rows:
        for col in cols:
            for value in values:
                if plp.value(target_vars[row][col][value]):
                    solution[row][col] = value # make it into an array

    return solution

### 3.2. Print the solution as a Sudoku grid

In [7]:
def print_solution(solution, rows,cols):
    # Print the final result
    print(f"\nFinal result:")

    print("\n\n+ ----------- + ----------- + ----------- +",end="")
    for row in rows:
        print("\n",end="\n|  ")
        for col in cols:
            num_end = "  |  " if ((col+1)%3 == 0) else "   "
            print(solution[row][col],end=num_end)

        if ((row+1)%3 == 0):
            print("\n\n+ ----------- + ----------- + ----------- +",end="")

## 4. Sudoku Solver <a name="solver"></a>

Sudoku Solver: Find a solution where the constraints are satisfied.   
Need to identify a feasible solution and not an optimal solution.

Decision Variables: 9x9x9 --> Binary variables: (row,column, value)   
Create a set of binary variables. 9 binary variables for each cell (row,col) - one for each value from 1 to 9.  
For every cell, only one of the 9 binary variables for that (row,col) can be set (constraint)

In [8]:
def solve_sudoku(input_sudoku, diagonal = False ):
    # Create the linear programming problem
    prob = plp.LpProblem("sudoku-solver")

    rows = range(0,9)
    cols = range(0,9)
    grids = range(0,9)
    values = range(1,10)

    # Decision Variable/Target variable
    target_vars = plp.LpVariable.dicts("grid_value", (rows,cols,values), cat='Binary') 

    # Set the objective function, needed for the PuLP but not for our problem
    objective = plp.lpSum(0)
    prob.setObjective(objective) 

    # Create the default constraints to solve sudoku
    add_default_constraints(prob, target_vars, rows, cols, grids, values)

    # Add the diagonal constraints if flag is set
    if diagonal:
        add_diagonal_sudoku_constraints(prob, target_vars, rows, cols, values)
        
    # Fill the prefilled values from input sudoku as constraints
    add_prefilled_constraints(prob, input_sudoku, target_vars, rows, cols, values)


    # Solve the problem
    prob.solve()

    # Print the status of the solution
    solution_status = plp.LpStatus[prob.status]
    print(f'Solution Status = {plp.LpStatus[prob.status]}')

    # Extract the solution if an optimal solution has been identified
    if solution_status == 'Optimal':
        solution = extract_solution(target_vars, rows, cols, values)
        print_solution(solution, rows,cols)

## 5. Random board creator <a name="random"></a>

### 5.1. C

## 6. Testing <a name="testing"></a>

### 6.1. Run the Normal Sudoku

In [12]:
normal_sudoku = [
                    [8,0,0,0,0,0,0,0,0],
                    [0,0,3,0,0,0,0,0,0],
                    [0,7,0,0,0,0,2,0,0],
                    [0,5,0,0,0,0,0,0,0],
                    [0,0,0,0,4,0,7,0,0],
                    [0,0,0,1,0,0,0,3,0],
                    [0,0,1,0,0,0,0,6,8],
                    [0,0,8,5,0,0,0,1,0],
                    [0,9,0,0,0,0,4,0,0]
                ]

solve_sudoku(input_sudoku = normal_sudoku,
             diagonal = False)

Solution Status = Optimal

Final result:


+ ----------- + ----------- + ----------- +

|  8   2   5  |  9   3   7  |  1   4   6  |  

|  4   1   3  |  2   5   6  |  8   9   7  |  

|  9   7   6  |  4   8   1  |  2   5   3  |  

+ ----------- + ----------- + ----------- +

|  1   5   4  |  3   7   2  |  6   8   9  |  

|  3   8   9  |  6   4   5  |  7   2   1  |  

|  2   6   7  |  1   9   8  |  5   3   4  |  

+ ----------- + ----------- + ----------- +

|  5   3   1  |  7   2   4  |  9   6   8  |  

|  7   4   8  |  5   6   9  |  3   1   2  |  

|  6   9   2  |  8   1   3  |  4   7   5  |  

+ ----------- + ----------- + ----------- +

### 6.2. Run the Diagonal Sudoku

In [10]:
diagonal_sudoku = [
                    [0,3,0,2,7,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0],
                    [8,0,0,0,0,0,0,0,0],
                    [5,1,0,0,0,0,0,8,4],
                    [4,0,0,5,9,0,0,7,0],
                    [2,9,0,0,0,0,0,1,0],
                    [0,0,0,0,0,0,1,0,5],
                    [0,0,6,3,0,8,0,0,7],
                    [0,0,0,0,0,0,3,0,0]
                  ]

solve_sudoku(input_sudoku = diagonal_sudoku,
             diagonal = True)

Solution Status = Optimal

Final result:


+ ----------- + ----------- + ----------- +

|  6   3   9  |  2   7   4  |  8   5   1  |  

|  1   4   2  |  6   8   5  |  7   3   9  |  

|  8   7   5  |  1   3   9  |  6   4   2  |  

+ ----------- + ----------- + ----------- +

|  5   1   3  |  7   6   2  |  9   8   4  |  

|  4   6   8  |  5   9   1  |  2   7   3  |  

|  2   9   7  |  8   4   3  |  5   1   6  |  

+ ----------- + ----------- + ----------- +

|  3   8   4  |  9   2   7  |  1   6   5  |  

|  9   5   6  |  3   1   8  |  4   2   7  |  

|  7   2   1  |  4   5   6  |  3   9   8  |  

+ ----------- + ----------- + ----------- +